<a href="https://colab.research.google.com/github/seohyun126/LG-Aimers-6th/blob/main/Aimers%EB%AA%A8%EB%8D%B8%EB%A7%813%EC%B0%A8_IVF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **데이터 확인**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
train=pd.read_csv('/content/drive/MyDrive/Aimers_data/train_IVF2.csv')
test=pd.read_csv('/content/drive/MyDrive/Aimers_data/test_IVF2.csv')

In [ ]:
test2=pd.read_csv("/content/drive/MyDrive/Aimers_data/test.csv")

In [ ]:
train.head()

,임신 시도 또는 마지막 임신 경과 연수,배란 자극 여부,단일 배아 이식 여부,불명확 불임 원인,불임 원인 - 난관 질환,불임 원인 - 남성 요인,불임 원인 - 배란 장애,불임 원인 - 자궁경부 문제,불임 원인 - 자궁내막증,불임 원인 - 정자 농도,...,혼합 이식 배아 수,남성 불임 원인,여성 불임 원인,정자 출처_배우자제공,정자 출처_기증제공,정자 보정된 나이,난자 기증 여부,현재 시술용 여부,사용된 신선 난자 수,동결 배아 이식 수
0,0.0,1,0.0,0,0,1,1,0,0,0,...,0.0,0,0,1,0,1.0,0.0,1.0,7.0,0.0
1,0.0,0,0.0,0,0,0,0,0,0,0,...,0.0,0,0,1,0,6.0,0.0,1.0,1.0,0.0
2,0.0,1,0.0,0,0,1,0,0,0,0,...,2.0,0,0,1,0,1.0,0.0,1.0,8.0,0.0
3,0.0,1,0.0,0,0,1,0,0,0,0,...,0.0,0,0,1,0,2.0,0.0,1.0,5.0,0.0
4,0.0,1,0.0,0,1,0,0,0,1,0,...,0.0,0,0,1,0,1.0,0.0,1.0,7.0,0.0


In [ ]:
num_cols=train.select_dtypes(exclude=['object']).columns
corr_values=train[num_cols].corr()['배아 이식 경과일']
corr_values[(corr_values >= 0.5) | (corr_values <= -0.5)]

,배아 이식 경과일
배란 자극 여부,0.586444
동결 배아 사용 여부,-0.690094
신선 배아 사용 여부,0.691271
난자 혼합 경과일,-0.686932
배아 이식 경과일,1.000000
난자배아성공률,0.634862
총 배아 수,0.562508
사용된 신선 난자 수,0.576130


In [ ]:
train.groupby('배아 이식 경과일')['임신 성공 여부'].mean()

,임신 성공 여부
배아 이식 경과일,
0.0,0.251014
1.0,0.187015
2.0,0.212469
3.0,0.258770
4.0,0.344361
5.0,0.404454
6.0,0.300036
7.0,0.411111


In [ ]:
corr_values[(corr_values >= 0.5) | (corr_values <= -0.5)].index

Index(['배란 자극 여부', '동결 배아 사용 여부', '신선 배아 사용 여부', '난자 혼합 경과일', '배아 이식 경과일',
       '난자배아성공률', '총 배아 수', '사용된 신선 난자 수'],
      dtype='object')

## **KNN Imputation -> 배아이식 경과일**

In [ ]:
# 상관관계가 높은 feature들로 채워줌
from sklearn.impute import KNNImputer

# 사용할 컬럼 선택
selected_columns = ['배란 자극 여부', '동결 배아 사용 여부', '신선 배아 사용 여부', '난자 혼합 경과일', '배아 이식 경과일',
       '난자배아성공률', '총 배아 수', '사용된 신선 난자 수']
# 선택한 컬럼만 포함한 데이터프레임 생성
train_selected = train[selected_columns]
# KNN Imputer 설정
imputer = KNNImputer(n_neighbors=5)  # K값은 5로 설정
train_selected_imputed = pd.DataFrame(imputer.fit_transform(train_selected), columns=selected_columns)

In [ ]:
# 원본 데이터프레임에 채워진 값 반영
train[selected_columns] = train_selected_imputed

In [ ]:
# 선택한 컬럼만 포함한 데이터프레임 생성
test_selected = test[selected_columns]
# KNN Imputer 설정
imputer = KNNImputer(n_neighbors=3)  # K값은 3로 설정
test_selected_imputed = pd.DataFrame(imputer.fit_transform(test_selected), columns=selected_columns)
# 원본 데이터프레임에 채워진 값 반영
test[selected_columns] = test_selected_imputed

In [ ]:
train.groupby('배아 이식 경과일')['임신 성공 여부'].mean().sort_values()

,임신 성공 여부
배아 이식 경과일,
5.8,0.000000
0.6,0.000000
2.8,0.001710
2.4,0.002029
2.2,0.002182
3.2,0.002608
2.6,0.003153
4.6,0.003880
4.4,0.005576


In [ ]:
train.drop(['IUI','Generic DI','IVI'],axis=1,inplace=True)
test.drop(['IUI','Generic DI','IVI'],axis=1,inplace=True)

In [ ]:
test.shape

(87891, 45)

In [ ]:
train.to_csv('/content/drive/MyDrive/Aimers_data/train_IVF2.csv',index=False)
test.to_csv('/content/drive/MyDrive/Aimers_data/test_IVF2.csv',index=False)

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.3 MB/s eta 0:00:00


## **모델링을 위한 데이터 준비**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from hyperopt import hp, fmin, tpe, Trials
from sklearn.metrics import roc_auc_score, make_scorer
#  데이터 준비
X = train.drop('임신 성공 여부',axis=1,inplace=False)
y = train['임신 성공 여부']  # 타겟 변수

## 📌 **Optuna + CatBoost + Train-Test Split + ROC-AUC**



In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import make_scorer, roc_auc_score
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
import numpy as np

# 범주형 변수 처리
cat_features = X.loc[:, (X.min() == 0) & (X.max() == 1)].columns.tolist()
X[cat_features] = X[cat_features].astype(int)

# 불균형 데이터 고려한 train-test split (Stratified)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# 클래스 불균형 처리
class_weights = {0: 1, 1: sum(y == 0) / sum(y == 1)}

# Extra Trees 모델 설정
model = ExtraTreesClassifier(random_state=42, n_jobs=-1, class_weight=class_weights)

# 하이퍼파라미터 탐색 공간 설정
param_dist = {
    'n_estimators': [100, 200, 500],
    'max_depth': [2, 4, 6, 8, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}


# ROC-AUC를 평가 지표로 설정
roc_auc_scorer = make_scorer(roc_auc_score)

In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.5 MB/s eta 0:00:00


In [ ]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

# 데이터 분할 (Train-Test Split)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Optuna Objective 함수 정의
def objective(trial):
    # 하이퍼파라미터 탐색 공간 설정
    params = {
        "iterations": trial.suggest_int("iterations", 100, 500),
        "depth": trial.suggest_int("depth", 2, 10),
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.1),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 1, 10),
        "one_hot_max_size": trial.suggest_int("one_hot_max_size", 2, 5),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "No"]),
        "random_strength": trial.suggest_uniform("random_strength", 0, 5),
        "bagging_temperature": trial.suggest_uniform("bagging_temperature", 0, 1),
        "class_weights": "Balanced",  # 불균형 데이터 처리
        "task_type": "CPU",
        "verbose": 0
    }

    # 모델 학습
    model = CatBoostClassifier(random_state=42, silent=True, task_type='CPU',cat_features=cat_features,class_weights=class_weights)
    model.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=50, verbose=3,)
    # 검증 데이터에서 ROC-AUC 평가
    y_pred = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, y_pred)
    print(f"Validation ROC-AUC Score: {auc:.4f}")  # 각 trial마다 ROC-AUC 출력

    return auc

In [ ]:
# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction="maximize")  # ROC-AUC 최대화
study.optimize(objective, n_trials=15, show_progress_bar=True)

# 최적의 하이퍼파라미터 출력
print("\nBest hyperparameters:", study.best_params)
print("Best ROC-AUC Score:", study.best_value)

[I 2025-02-18 09:55:43,939] A new study created in memory with name: no-name-2116881c-1ceb-4926-8031-b0a9bafa21a2


  0%|          | 0/15 [00:00<?, ?it/s]

<ipython-input-54-f74980a0dd0c>:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.1),
<ipython-input-54-f74980a0dd0c>:21: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "random_strength": trial.suggest_uniform("random_strength", 0, 5),
<ipython-input-54-f74980a0dd0c>:22: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "bagging_temperature": trial.suggest_uniform("bagging_temperature", 0, 1),


Learning rate set to 0.117297
0:	learn: 0.6662518	test: 0.6662755	best: 0.6662755 (0)	total: 63.1ms	remaining: 1m 3s
3:	learn: 0.6217984	test: 0.6223456	best: 0.6223456 (3)	total: 207ms	remaining: 51.6s
6:	learn: 0.6037571	test: 0.6046257	best: 0.6046257 (6)	total: 329ms	remaining: 46.7s
9:	learn: 0.5986622	test: 0.5995518	best: 0.5995518 (9)	total: 445ms	remaining: 44s
12:	learn: 0.5895027	test: 0.5907758	best: 0.5907758 (12)	total: 561ms	remaining: 42.6s
15:	learn: 0.5876150	test: 0.5889584	best: 0.5889584 (15)	total: 679ms	remaining: 41.8s
18:	learn: 0.5842883	test: 0.5861070	best: 0.5861070 (18)	total: 805ms	remaining: 41.6s
21:	learn: 0.5833439	test: 0.5852510	best: 0.5852510 (21)	total: 919ms	remaining: 40.8s
24:	learn: 0.5826249	test: 0.5847235	best: 0.5847235 (24)	total: 1.03s	remaining: 40.3s
27:	learn: 0.5820280	test: 0.5841848	best: 0.5841848 (27)	total: 1.15s	remaining: 39.8s
30:	learn: 0.5813900	test: 0.5837612	best: 0.5837612 (30)	total: 1.28s	remaining: 40.2s
33:	learn: 

<ipython-input-54-f74980a0dd0c>:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.1),
<ipython-input-54-f74980a0dd0c>:21: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "random_strength": trial.suggest_uniform("random_strength", 0, 5),
<ipython-input-54-f74980a0dd0c>:22: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "bagging_temperature": trial.suggest_uniform("bagging_temperature", 0, 1),


...

Learning rate set to 0.117297
0:	learn: 0.6662518	test: 0.6662755	best: 0.6662755 (0)	total: 36.4ms	remaining: 36.4s
3:	learn: 0.6217984	test: 0.6223456	best: 0.6223456 (3)	total: 150ms	remaining: 37.3s
6:	learn: 0.6037571	test: 0.6046257	best: 0.6046257 (6)	total: 275ms	remaining: 39s
9:	learn: 0.5986622	test: 0.5995518	best: 0.5995518 (9)	total: 387ms	remaining: 38.3s
12:	learn: 0.5895027	test: 0.5907758	best: 0.5907758 (12)	total: 507ms	remaining: 38.5s
15:	learn: 0.5876150	test: 0.5889584	best: 0.5889584 (15)	total: 623ms	remaining: 38.3s
18:	learn: 0.5842883	test: 0.5861070	best: 0.5861070 (18)	total: 744ms	remaining: 38.4s
21:	learn: 0.5833439	test: 0.5852510	best: 0.5852510 (21)	total: 860ms	remaining: 38.2s
24:	learn: 0.5826249	test: 0.5847235	best: 0.5847235 (24)	total: 986ms	remaining: 38.5s
27:	learn: 0.5820280	test: 0.5841848	best: 0.5841848 (27)	total: 1.11s	remaining: 38.4s
30:	learn: 0.5813900	test: 0.5837612	best: 0.5837612 (30)	total: 1.23s	remaining: 38.3s
33:	learn: 

<ipython-input-54-f74980a0dd0c>:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.1),
<ipython-input-54-f74980a0dd0c>:21: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "random_strength": trial.suggest_uniform("random_strength", 0, 5),
<ipython-input-54-f74980a0dd0c>:22: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "bagging_temperature": trial.suggest_uniform("bagging_temperature", 0, 1),


Learning rate set to 0.117297
0:	learn: 0.6662518	test: 0.6662755	best: 0.6662755 (0)	total: 49.1ms	remaining: 49.1s
3:	learn: 0.6217984	test: 0.6223456	best: 0.6223456 (3)	total: 258ms	remaining: 1m 4s
6:	learn: 0.6037571	test: 0.6046257	best: 0.6046257 (6)	total: 502ms	remaining: 1m 11s
9:	learn: 0.5986622	test: 0.5995518	best: 0.5995518 (9)	total: 693ms	remaining: 1m 8s
12:	learn: 0.5895027	test: 0.5907758	best: 0.5907758 (12)	total: 814ms	remaining: 1m 1s
15:	learn: 0.5876150	test: 0.5889584	best: 0.5889584 (15)	total: 923ms	remaining: 56.8s
18:	learn: 0.5842883	test: 0.5861070	best: 0.5861070 (18)	total: 1.05s	remaining: 54.1s
21:	learn: 0.5833439	test: 0.5852510	best: 0.5852510 (21)	total: 1.16s	remaining: 51.5s
24:	learn: 0.5826249	test: 0.5847235	best: 0.5847235 (24)	total: 1.3s	remaining: 50.6s
27:	learn: 0.5820280	test: 0.5841848	best: 0.5841848 (27)	total: 1.4s	remaining: 48.8s
30:	learn: 0.5813900	test: 0.5837612	best: 0.5837612 (30)	total: 1.51s	remaining: 47.3s
33:	learn:

<ipython-input-54-f74980a0dd0c>:17: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.1),
<ipython-input-54-f74980a0dd0c>:21: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "random_strength": trial.suggest_uniform("random_strength", 0, 5),
<ipython-input-54-f74980a0dd0c>:22: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "bagging_temperature": trial.suggest_uniform("bagging_temperature", 0, 1),


Learning rate set to 0.117297
0:	learn: 0.6662518	test: 0.6662755	best: 0.6662755 (0)	total: 37.2ms	remaining: 37.2s
3:	learn: 0.6217984	test: 0.6223456	best: 0.6223456 (3)	total: 151ms	remaining: 37.5s
6:	learn: 0.6037571	test: 0.6046257	best: 0.6046257 (6)	total: 271ms	remaining: 38.5s
9:	learn: 0.5986622	test: 0.5995518	best: 0.5995518 (9)	total: 389ms	remaining: 38.5s
12:	learn: 0.5895027	test: 0.5907758	best: 0.5907758 (12)	total: 515ms	remaining: 39.1s
15:	learn: 0.5876150	test: 0.5889584	best: 0.5889584 (15)	total: 624ms	remaining: 38.4s
18:	learn: 0.5842883	test: 0.5861070	best: 0.5861070 (18)	total: 757ms	remaining: 39.1s
21:	learn: 0.5833439	test: 0.5852510	best: 0.5852510 (21)	total: 873ms	remaining: 38.8s
24:	learn: 0.5826249	test: 0.5847235	best: 0.5847235 (24)	total: 993ms	remaining: 38.7s
27:	learn: 0.5820280	test: 0.5841848	best: 0.5841848 (27)	total: 1.11s	remaining: 38.5s
30:	learn: 0.5813900	test: 0.5837612	best: 0.5837612 (30)	total: 1.22s	remaining: 38.1s
33:	learn

In [ ]:
# 최적의 하이퍼파라미터로 전체 데이터 학습
best_params = study.best_params  # 최적 하이퍼파라미터 가져오기

print("\nBest hyperparameters:", best_params)
print("Best ROC-AUC Score:", study.best_value)


Best hyperparameters: {'iterations': 200, 'depth': 9, 'learning_rate': 0.021621181278789155, 'l2_leaf_reg': 8, 'one_hot_max_size': 4, 'bootstrap_type': 'Bayesian', 'random_strength': 4.23790289814099, 'bagging_temperature': 0.49691320935503247}
Best ROC-AUC Score: 0.7306383356508922


In [ ]:
# 최적의 하이퍼파라미터를 사용하여 최종 모델 학습 (전체 데이터 X, y 사용)
final_model = CatBoostClassifier(**best_params, random_state=42, class_weights=class_weights, task_type="CPU",cat_features=cat_features)

In [ ]:
final_model.fit(X,y)

0:	learn: 0.6880737	total: 133ms	remaining: 26.5s
1:	learn: 0.6834370	total: 189ms	remaining: 18.7s
2:	learn: 0.6766291	total: 261ms	remaining: 17.1s
3:	learn: 0.6703574	total: 337ms	remaining: 16.5s
4:	learn: 0.6662589	total: 402ms	remaining: 15.7s
5:	learn: 0.6633362	total: 468ms	remaining: 15.1s
6:	learn: 0.6606146	total: 529ms	remaining: 14.6s
7:	learn: 0.6580898	total: 597ms	remaining: 14.3s
8:	learn: 0.6558709	total: 664ms	remaining: 14.1s
9:	learn: 0.6538484	total: 729ms	remaining: 13.8s
10:	learn: 0.6493163	total: 824ms	remaining: 14.2s
11:	learn: 0.6467551	total: 952ms	remaining: 14.9s
12:	learn: 0.6450260	total: 1.04s	remaining: 15s
13:	learn: 0.6430240	total: 1.15s	remaining: 15.2s
14:	learn: 0.6414088	total: 1.28s	remaining: 15.8s
15:	learn: 0.6376018	total: 1.41s	remaining: 16.2s
16:	learn: 0.6340304	total: 1.54s	remaining: 16.6s
17:	learn: 0.6331727	total: 1.66s	remaining: 16.8s
18:	learn: 0.6300202	total: 1.78s	remaining: 17s
19:	learn: 0.6270904	total: 1.92s	remaining: 

In [ ]:
X.shape

(250054, 45)

In [ ]:
test.shape

(87891, 45)

In [ ]:
feature_importance = final_model.get_feature_importance()

# feature importance를 DataFrame으로 변환
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importance
})

# feature importance 내림차순 정렬
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)


In [ ]:
feature_importance_df

,Feature,Importance
35,혼합 이식 배아 수,35.628867
19,미세주입 배아 이식 수,30.433785
42,현재 시술용 여부,14.305386
34,총 배아 수,4.564038
24,배아 이식 경과일,3.872349
25,보정된 나이,3.498873
27,IVF,1.579962
26,ICSI,1.369929
33,난자배아성공률,0.762055
40,정자 보정된 나이,0.662478


In [ ]:
test[cat_features]=test[cat_features].astype(int)
cat_test_preds = final_model.predict_proba(test)[:, 1]
test['임신 성공 여부'] = cat_test_preds

In [ ]:
test.shape

(87891, 46)

In [ ]:
test2.shape

(90067, 68)

In [ ]:
test['ID']=test2['ID'].reset_index(drop=True)

In [ ]:
sub=pd.read_csv("/content/drive/MyDrive/Aimers_data/sample_submission3_2.csv",index_col=False)

In [ ]:
# 'test'와 'sub' 데이터를 ID를 기준으로 병합, sub 크기를 유지하도록 left join
merged = sub.merge(test[['ID', '임신 성공 여부']], on='ID', how='left')

# '임신 성공 여부' 값이 있는 경우에만 'prob'을 업데이트
merged['probability'] = merged['임신 성공 여부'].fillna(merged['probability'])

In [ ]:
merged.drop('임신 성공 여부',axis=1,inplace=True)

In [ ]:
merged.to_csv("/content/drive/MyDrive/Aimers_data/sample_submission4_1.csv",index=False)

In [ ]:
from tqdm import tqdm
import random
# 랜덤 샘플링된 하이퍼파라미터 리스트 생성
n_iter = 15
# 하이퍼파라미터 탐색 공간 설정
param_dist = {
    'n_estimators': [100, 200, 500],
    'max_depth': [2, 4, 6, 8, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}

# 랜덤 샘플링된 하이퍼파라미터 리스트 생성
n_iter = 15
param_list = []
for _ in range(n_iter):
    param_sample = {key: random.choice(values) for key, values in param_dist.items()}
    param_list.append(param_sample)

# tqdm으로 진행률 표시
for i, params in enumerate(tqdm(param_list, desc="Hyperparameter Search")):
    model.set_params(**params)
    model.fit(X_train, y_train)

    # 검증 데이터에서 ROC-AUC 평가
    y_pred = model.predict_proba(X_valid)[:, 1]
    auc = roc_auc_score(y_valid, y_pred)

    print(f"Iteration {i+1} - ROC-AUC Score: {auc:.4f} - Params: {params}")

Hyperparameter Search:   7%|▋         | 1/15 [00:21<05:05, 21.86s/it]

Iteration 1 - ROC-AUC Score: 0.7214 - Params: {'n_estimators': 100, 'max_depth': None, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'log2', 'bootstrap': False}


Hyperparameter Search:  13%|█▎        | 2/15 [00:25<02:28, 11.44s/it]

Iteration 2 - ROC-AUC Score: 0.7050 - Params: {'n_estimators': 100, 'max_depth': 6, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'log2', 'bootstrap': True}


Hyperparameter Search:  20%|██        | 3/15 [02:57<15:07, 75.61s/it]

...

Iteration 13 - ROC-AUC Score: 0.7081 - Params: {'n_estimators': 200, 'max_depth': 6, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'bootstrap': False}


Hyperparameter Search:  93%|█████████▎| 14/15 [09:32<00:30, 30.33s/it]

Iteration 14 - ROC-AUC Score: 0.7074 - Params: {'n_estimators': 100, 'max_depth': None, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'bootstrap': True}


Hyperparameter Search: 100%|██████████| 15/15 [10:27<00:00, 41.84s/it]

Iteration 15 - ROC-AUC Score: 0.6986 - Params: {'n_estimators': 200, 'max_depth': None, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'bootstrap': False}


In [ ]:
# 최적 파라미터 출력
print("Best parameters found: ", model.best_params_)
# 최적 모델
best_model = random_search.best_estimator_

AttributeError: 'ExtraTreesClassifier' object has no attribute 'best_params_'